<a href="https://colab.research.google.com/github/Taemin11/bigdata/blob/master/database_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
##구글드라이브 권한 획득
!apt install -y -qq --fix-broken python-pycurl python-apt 2> /dev/null
!apt install -y -qq software-properties-common module-init-tools 2> /dev/null
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt update -qq 2>&1 > /dev/null
!apt install -y -qq google-drive-ocamlfuse fuse 2> /dev/null

from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
creds = GoogleCredentials.get_application_default()

!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL

from getpass import getpass

vcode = getpass()

!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}



The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
Suggested packages:
  python-apt-dbg python-apt-doc libcurl4-gnutls-dev python-pycurl-dbg
  python-pycurl-doc
The following NEW packages will be installed:
  python-apt python-pycurl
0 upgraded, 2 newly installed, 0 to remove and 8 not upgraded.
Need to get 193 kB of archives.
After this operation, 861 kB of additional disk space will be used.
Selecting previously unselected package python-apt.
(Reading database ... 130912 files and directories currently installed.)
Preparing to unpack .../python-apt_1.6.4_amd64.deb ...
Unpacking python-apt (1.6.4) ...
Selecting previously unselected package python-pycurl.
Preparing to unpack .../python-pycurl_7.43.0.1-0.2_amd64.deb ...
Unpacking python-pycurl (7.43.0.1-0.2) ...
Setting up python-pycurl (7.43.0.1-0.2) ...
Setting up python-apt (1.6.4) ...
software-properties-common is already the newest version (0.96.24

In [0]:
#drive에 경로 생성
!mkdir -p google_drive
!google-drive-ocamlfuse -o nonempty google_drive

In [0]:
#셀레니옴 설치
# install chromium, its driver, and selenium
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium
# set options to be headless, ..
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
# open it, go to a website, and get results
wd = webdriver.Chrome('chromedriver',options=options)
wd.get("https://www.website.com")
print(wd.page_source)  # results

In [0]:
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import numpy as np
import requests
import itertools
import hashlib 

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')



driver = webdriver.Chrome('chromedriver',options=options)

cols = "rank", "type", "gameName", "companyName", "imgURL", "date", "appID", "market", "companyId"
df = pd.DataFrame(columns = cols)
nn = 0
driver.get('http://www.gevolution.co.kr/rank/history.asp')
driver.find_element_by_xpath('//*[@id="btn_loginLayer"]/a').click()
elem_login = driver.find_element_by_id("userLoginID")
elem_login.send_keys('ht4380')
elem_login = driver.find_element_by_id("userPassWd")
elem_login.send_keys('14elqlphp')
driver.find_element_by_xpath('//*[@id="loginSec"]/div[1]/fieldset/div[3]/button').click()
for uu in range(1,3):
  kkk = 19  
  driver.find_element_by_xpath('//*[@id="txtStore"]/option['+str(uu)+']').click()
  print("go")
  if uu == 1:
    market = '구글플레이'
  elif uu == 2:
    market = '앱스토어'
  elif uu == 3:
    market = '원스토어'
  for gg in range(1,8):
    driver.find_element_by_xpath('//*[@id="schDate"]').click()
    driver.find_element_by_xpath('//*[@id="ui-datepicker-div"]/div/div/select[2]/option[5]').click()
    driver.find_element_by_xpath('//*[@id="ui-datepicker-div"]/table/tbody/tr[4]/td['+str(gg)+']/a').click()
    source = driver.find_element_by_xpath('//*[@id="imgload"]/table/tbody')
    ###이미지 크롤링
    req = driver.page_source
    soup = BeautifulSoup(req, "html.parser")
    #print(soup)
    aaa = []
    #a = 0
    bbb = []
    #b = 0
    ccc = []
    #c = 0
    i = 0
    for img in soup.find_all("img"):
      imgsrc = img.get('original')
      #print(img.get('original'))
      if imgsrc is None:
        print("none")
      else: 
        if i%3 == 0:
          aaa.append(imgsrc)
        elif i%3 == 1:
          bbb.append(imgsrc)
        elif i%3 == 2:
          ccc.append(imgsrc)
        i+=1

    ###텍스트 크롤링 
    sss = source.text
    ttt = []
    ttt.append(sss.split("\n")) 
    for i in range(0, 700):
      if "비트 MP3 for 유튜브" in ttt[0][i]:
        ttt[0].insert(i+1,"youtube")      
    aa = [[0]*11 for m in range(100)]
    bb = [[0]*11 for m in range(100)]
    cc = [[0]*11 for m in range(100)]
    for j in range(0,100):
      #무료순위
      aa[j][0] = ttt[0][j*7]
      aa[j][1] = 0
      aa[j][2] = ttt[0][j*7 + 1]
      aa[j][3] = ttt[0][j*7 + 2]    
      aa[j][4] = aaa[j]
      aa[j][5] = "2019-05-" + str(kkk)
      aa[j][6] = (hash(aaa[j][59:67]) % 10**8)
      aa[j][7] = market
      aa[j][8] = (hash(ttt[0][j*7 + 2]) % 10**8)
      #유료순위
      bb[j][0] = ttt[0][j*7]
      bb[j][1] = 1
      bb[j][2] = ttt[0][j*7 + 3]
      bb[j][3] = ttt[0][j*7 + 4]
      bb[j][4] = bbb[j]
      bb[j][5] = "2019-05-" + str(kkk)
      bb[j][6] = (hash(bbb[j][59:67]) % 10**8)
      bb[j][7] = market
      bb[j][8] = (hash(ttt[0][j*7 + 4]) % 10**8)
      #매출순위
      cc[j][0] = ttt[0][j*7]
      cc[j][1] = 2
      cc[j][2] = ttt[0][j*7 + 5]
      cc[j][3] = ttt[0][j*7 + 6]
      print(ttt[0][j*7 + 1])
      print(ttt[0][j*7 + 2])
      print(ttt[0][j*7 + 3])
      print(ttt[0][j*7 + 4])
      print(ttt[0][j*7 + 5])
      print(ttt[0][j*7 + 6])
      cc[j][4] = ccc[j]
      cc[j][5] = "2019-05-" + str(kkk)
      cc[j][6] = (hash(ccc[j][59:67]) % 10**8)
      cc[j][7] = market
      cc[j][8] = (hash(ttt[0][j*7 + 6]) % 10**8)
    print("kkk : ",  kkk)
    kkk += 1

    for n in range(0,100):
      df.loc[nn] = aa[n]
      nn +=1
    for n in range(0,100):
      df.loc[nn] = bb[n]
      nn +=1    
    for n in range(0,100):
      df.loc[nn] = cc[n]
      nn +=1  

!mkdir -p google_drive/test_colab
df.to_csv("google_drive/test_colab/Rawdata_gevolutiontest.csv")
!ls -Fcal google_drive/test_colab

#sss = sss.split()


ModuleNotFoundError: ignored

In [0]:
#xml파싱
import pandas as pd
from urllib.request import urlopen
import xml.etree.ElementTree as et

!mkdir -p google_drive
!google-drive-ocamlfuse -o nonempty google_drive

for i in range(1, 4):
  xml = urlopen('http://api.gevolution.co.kr/rank/xml/?aCode=MNOP803974&market=a&appType=game&rankType='+str(i)+'&rank=100')
  xtree = et.parse(xml)
  xroot = xtree.getroot()

  aid = ""
  ranking = ""
  lastWeek = ""
  gameName = ""
  publisher = ""
  iconUrl = ""

  for item in xroot.iter("item"):
    aid += item.findtext("aid") + "^"
    ranking += item.findtext("ranking") + "^"
    lastWeek += item.findtext("lastWeek") + "^"
    gameName += item.findtext("gameName") + "^"
    publisher += item.findtext("publisher") + "^"
    iconUrl += item.findtext("iconUrl") + "^"


  aid = aid.split("^")
  ranking = ranking.split("^")
  lastWeek = lastWeek.split("^")
  gameName = gameName.split("^")
  publisher = publisher.split("^")
  iconUrl = iconUrl.split("^")


  data = {'aid' : aid,
         'ranking' : ranking,
         'lastWeek' : lastWeek,
         'gameName' : gameName,
         'publisher' : publisher,
         'iconUrl' : iconUrl}



!mkdir -p google_drive/test_colab
df = pd.DataFrame(data)
df.to_csv("google_drive/test_colab/asssddddsssdddd.csv")
!ls -Fcal google_drive/test_colab

In [0]:

qqq = 'http://upload.gevolution.co.kr/appicon/s0610000/gevolution_00609871_icon_50.png'
qqq[59:67]

'00609871'